In [2]:
import numpy as np
import sklearn
from skimage.measure import block_reduce
from PIL import Image
import matplotlib.pyplot as plt
from PIL import ImageFilter
import tensorflow
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_folder = 'shapes'
img_size = (128, 128)
batch_size = 30

datagen = ImageDataGenerator(rescale=1./255, # normalize pixel values
                             validation_split=0.3) # hold back 30% of the images for validation

train_generator = datagen.flow_from_directory(
    data_folder,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = datagen.flow_from_directory(
    data_folder,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

classnames = list(train_generator.class_indices.keys())

# Define the model as a sequence of layers
model = Sequential()

# The input layer accepts an image and applies a convolution that uses 32 6x6 filters and a rectified linear unit activation function
model.add(Conv2D(32, (6, 6), input_shape=train_generator.image_shape, activation='relu'))

# Next we;ll add a max pooling layer with a 2x2 patch
model.add(MaxPooling2D(pool_size=(2,2)))

# We can add as many layers as we think necessary - here we'll add another convolution, max pooling, and dropout layer
model.add(Conv2D(32, (6, 6), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# And another set
model.add(Conv2D(32, (6, 6), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# A dropout layer randomly drops some nodes to reduce inter-dependencies (which can cause over-fitting)
model.add(Dropout(0.2))

# Now we'll flatten the feature maps and generate an output layer with a predicted probability for each class
model.add(Flatten())
model.add(Dense(train_generator.num_classes, activation='sigmoid'))

# With the layers defined, we can now compile the model for categorical (multi-class) classification
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model over 3 epochs using 30-image batches and using the validation holdout dataset for validation
num_epochs = 5
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = num_epochs)




Found 840 images belonging to 3 classes.
Found 360 images belonging to 3 classes.
Epoch 1/5
28/28 [==============================] - 41s 1s/step - loss: 0.7396 - acc: 0.6321 - val_loss: 0.3950 - val_acc: 0.7778
Epoch 2/5
28/28 [==============================] - 40s 1s/step - loss: 0.2431 - acc: 0.8917 - val_loss: 0.0820 - val_acc: 0.9611
Epoch 3/5
28/28 [==============================] - 40s 1s/step - loss: 0.0419 - acc: 0.9857 - val_loss: 0.0273 - val_acc: 0.9889
Epoch 4/5
28/28 [==============================] - 41s 1s/step - loss: 0.0087 - acc: 0.9976 - val_loss: 0.0022 - val_acc: 1.0000
Epoch 5/5
28/28 [==============================] - 40s 1s/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.0015 - val_acc: 1.0000
